In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import load_component
from azure.ai.ml.entities import PipelineComponentBatchDeployment

# Connect to the workspace
credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")

ml_client = MLClient.from_config(credential=credential)

insurance_model_training_reg = load_component(source="Train_reg_pipeline.yml")
registered_training_pipeline = ml_client.components.create_or_update(insurance_model_training_reg)

latest_component_version = max([(m.version) for m in ml_client.components.list(name="traininig_registeration_pipeline")])
registered_training_pipeline = ml_client.components.get(name="traininig_registeration_pipeline", version=latest_component_version)


deployment = PipelineComponentBatchDeployment(
    name="training-pipeline",
    description="Deploying pipeline for model training and registration",
    endpoint_name="training-deployment-endpoint",
    component=registered_training_pipeline,
    settings={"continue_on_step_failure": False, "default_compute": "VM1"},
)
ml_client.batch_deployments.begin_create_or_update(deployment).result()